## 03 funcpp

### funcpp0

In [ ]:
from RZutilpy.system import Path, unix_wrapper, gettimestr, makedirs
from RZutilpy.rzio import matchfiles
from RZutilpy.mri import findminoutlier
from RZutilpy.figure import plot
import os
import numpy as np
import matplotlib.pyplot as plt

# ========================== Set parameter ============================
# subject, session, run
subj = 'CN040'
session = '20230619_CN040_session2/'
runlist = ['201','301','401','601','701','901','1001','1101','1301','1401','501','801','1201']
#runlist = ['301','401','501','701','801','1001','1101','1201','1401','1501','601','901','1301']
#runlist = ['201','301','401','601','701','1101','1201','1301','1501','1601','501','801','1401']
# =====================================================================


foldpath = '/home/data/rawdata/facePRF/'+subj+'_faceprf/'+session
os.chdir(foldpath+'rawdata/')

# a list of functional NIFTI files
files = []
for r in runlist:
    files.append(matchfiles(foldpath+'rawdata/'+'*_'+r+'_task*.nii'))


# t1 file
t1 = Path(f'/home/software/freesurfer/7.3.2/subjects/{subj}/SUMA/{subj}_SurfVol.nii') # 256x256x256
t1ssmask = Path(f'/home/data/rawdata/facePRF/{subj}_faceprf/20230607_CN040_session0/anatpp/T1_SSMask.nii')
t1ss = subj+'_SurfVol_SS.nii'

# output directory, if exists, we exit
output_dir = Path(foldpath+'/funcpp/')

# number of tr to discard
tr_discard = 0  # number of tr to discard

# motion censor limit
motion_censor = 0.3  # threshold for motion censoring, default:(0.3)

# extra option for align_epi_anat.py
align_opt = ['-giant_move']

# epi2anat(individual)
resolution = 2.5

# grid at each axies of {subj}.SurfVol.nii
grid_RL = 256
grid_AP = 256
grid_IS = 256

# some calculation
fwdnum = 10
revnum = 3
nRuns = len(files)
runstr = [f'{i:02d}AP' for i in range(1,fwdnum+1)] + [f'{i:03d}PA' for i in range(1,revnum+1)]
motion_censor = 0.3 if motion_censor is None else motion_censor

fwdfiles = ['01AP', '02AP','03AP', '04AP', '05AP', '06AP', '07AP', '08AP', '09AP', '10AP'] # 这里需要手动run的顺序，用来做配对distortion correction
revfiles = ['001PA','001PA','001PA', '002PA','002PA', '003PA','003PA','003PA','003PA','003PA']

In [ ]:
# ============== deal with some input parameters =======================
# Show some diagnostic information
# check file exists
for i in files:
    assert Path(i).exists(), f'{i} does not exist!'

t1 = Path(t1)
assert t1.exists(), 'T1 file does not exist!'

# generate slice timing file
if not Path('SliceTiming.txt').exists():
    import json 
    #with open(f'{Path(files[0])}.json') as f:
    with open(f'{files[0][:-4]}.json') as f:
        jsoninfo = json.load(f)
    np.savetxt('SliceTiming.txt', jsoninfo['SliceTiming'])
    print('generate SliceTiming.txt!')
    del jsoninfo

# 整理forward和reverse的文件，如果两者数量不相等，我们自动补齐缺数量
if len(fwdfiles)!=len(revfiles):
    if len(fwdfiles)>len(revfiles):
        nPair = len(fwdfiles)
        revfiles = revfiles + [revfiles[-1]]*(nPair-len(revfiles))
    else:
        nPair = len(revfiles)
        fwdfiles = fwdfiles + [fwdfiles[-1]]*(nPair-len(fwdfiles))

# print out some diagnoistic
print(f'\nt1 file is: \n{t1}')
print(f'functional files are: \n')
[print(f'{i}') for i in files]
print(f'\nforward files are: {fwdfiles} \n')
print(f'reverse files are: {revfiles} \n')
print(f'\nslice timing file is: \nSliceTiming.txt')



start_time = gettimestr("full")
print(f'\n=============== Preprocessing started: {start_time} ================\n')
# change shell to tcsh, afni default shell is tcsh
orig_shell = os.environ['SHELL']
os.environ['SHELL']='/usr/bin/tcsh'
cwd = Path.cwd() # record current directory, we will go back

# verify that the results directory does not yet exist
output_dir = Path(output_dir)
assert not output_dir.exists(), f'output dir {output_dir} already exists'
makedirs(output_dir)
# enter the results directory (can begin processing data)
os.chdir(output_dir)
# copy anatomy to results dir
! 3dcopy {t1} {t1.pstem}
t1 = Path(f'{t1.pstem}+orig')  # switch t1 to the new location

# copy nonlinear-warping MNI files to this folder
FREESURFER_HOME = os.getenv('FREESURFER_HOME')
AFNI_HOME = os.getenv('AFNIDIR')
! cp {FREESURFER_HOME}/subjects/{subj}/mni/anatQQ.{subj}_WARP.nii ./
! cp {FREESURFER_HOME}/subjects/{subj}/mni/anatQQ.{subj}.aff12.1D ./
! cp {AFNI_HOME}/MNI152_2009_template_SSW.nii.gz ./
! cp {t1ssmask} T1_SSMask.{subj}.nii
! 3dcopy {t1} {subj}_SurfVol.nii


# 根据T1_SSMask.{subj}.nii得到剥头皮后的T1，但首先要把朝向和grid对齐
! 3dresample -orient LPI -prefix {subj}_SurfVol_2std.nii -input {subj}_SurfVol.nii

#-------------------------------------------------------------
# 利用3dZeropad将mask和T1像的grid对齐，否则无法把mask和T1像相乘：
# 例如在这个例子中{subj}_SurfVol.nii是256 x 256 x 256，而T1_SSMask.{subj}.nii是240 x 256 x 256。所以这里在T1_SSMask.{subj}.nii的第一个轴（R-L）修改为256。相当于在左右两侧分别加了8.
# 3个轴： -RL -AP -IS，请根据具体情况修改此行代码
! 3dZeropad -RL {grid_RL} -prefix T1_SSMask_grid.{subj}.nii  T1_SSMask.{subj}.nii
! 3dZeropad -AP {grid_AP} -prefix T1_SSMask_grid.{subj}.nii  T1_SSMask_grid.{subj}.nii -overwrite
! 3dZeropad -IS {grid_IS} -prefix T1_SSMask_grid.{subj}.nii  T1_SSMask_grid.{subj}.nii -overwrite
#-------------------------------------------------------------

! 3dcalc -a {subj}_SurfVol_2std.nii -b T1_SSMask_grid.{subj}.nii -expr 'a*step(b)' -prefix {t1ss}
t1ss = Path(t1ss)  # switch t1 to the new location



# ============================ auto block: tcat ============================
# apply 3dTcat to copy input dsets to results dir,
# while might want removing the first TRs

for file, run in zip(files, runstr):
    cmd=f'3dTcat -prefix pb00.{subj}.r{run}.tcat {file}[{tr_discard}..$]'
    unix_wrapper(cmd)

# ===========================reorient =========================
# 利用3dresample把朝向划到标准朝向LPI
# 需要AP,PA一起做
for run in runstr:
    ! 3dresample -orient LPI -prefix pb00.{subj}.r{run}.tcat+orig -input pb00.{subj}.r{run}.tcat+orig -overwrite


# ============================auto block: giant_move, added by RZ ===========================
for run in runstr:
    # 把图像的中点划到统一显示空间的中点
    ! 3drefit -deoblique -xorigin cen -yorigin cen -zorigin cen pb00.{subj}.r{run}.tcat+orig
    # 然后把function数据和 t1ss文件的重心移动到一致，这样有利于进行配准
    ! @Align_Centers -cm -no_cp -base {t1ss} -dset pb00.{subj}.r{run}.tcat+orig

! rm *_shft.1D


findminoutlier(f'pb00.{subj}.r*AP.tcat+orig.HEAD', output_dir)    

### funcpp02

功能像预处理的最核心部分。这一部分我们完成一下步骤
* Despike
* Slice-timing correction (时间层矫正)
* Distortion correction (图像畸变矫正)
* Motion correction (头动矫正)
* align anat 2 epi (结构像到功能像的配准) 
* 把所有epi划到个体结构像空间
* 把所有epi的volume数据划到surface data
* 把所有epi的划到MNI标准空间
* Scale (标准化数据尺度，转换成percent signal change)

In [ ]:
# ================================ despike =================================
for run in runstr:
    ! 3dDespike -NEW -nomask -prefix pb01.{subj}.r{run}.despike pb00.{subj}.r{run}.tcat+orig
#! rm pb00*


# ================================= tshift =================================
for run in runstr:
    ! 3dTshift -tzero 0 -quintic -prefix pb02.{subj}.r{run}.tshift \
         -verbose -tpattern @{'../rawdata/SliceTiming.txt'} pb01.{subj}.r{run}.despike+orig

# 删掉上一步despike的data节省空间
! rm pb01*


In [ ]:
# ================================= distortion =================================
# 首先找到
for fwd,rev in zip(fwdfiles, revfiles):
        # create median datasets from forward and reverse time series
        ! 3dTstat -median -prefix rm.blip.med.fwd.r{fwd} pb02.{subj}.r{fwd}.tshift+orig
        ! 3dTstat -median -prefix rm.blip.med.rev.r{rev} pb02.{subj}.r{rev}.tshift+orig
        # automask the median datasets 
        ! 3dAutomask -apply_prefix rm.blip.med.masked.fwd.r{fwd} rm.blip.med.fwd.r{fwd}+orig
        ! 3dAutomask -apply_prefix rm.blip.med.masked.rev.r{rev} rm.blip.med.rev.r{rev}+orig
        # compute the midpoint warp between the median datasets
        ! 3dQwarp -plusminus -pmNAMES Rev.r{rev} Fwd.r{fwd}                           \
                -pblur 0.05 0.05 -blur -1 -1                          \
                -noweight -minpatch 9                                 \
                -noXdis -noZdis                                       \
                -source rm.blip.med.masked.rev.r{rev}+orig                   \
                -base   rm.blip.med.masked.fwd.r{fwd}+orig                   \
                -prefix blip_warp
        
        ! 3dNwarpApply -quintic -nwarp blip_warp_Fwd.r{fwd}_WARP+orig        \
                -source rm.blip.med.masked.fwd.r{fwd}+orig               \
                -prefix rm.blip.med.masked.fwd.post.r{fwd}

        ! 3dNwarpApply -quintic -nwarp blip_warp_Rev.r{rev}_WARP+orig        \
                -source rm.blip.med.masked.rev.r{rev}+orig               \
                -prefix rm.blip.med.masked.rev.post.r{rev}
        
        # 删掉多余文件
        ! rm blip_warp_Fwd.r{fwd}+orig* blip_warp_Rev.r{rev}+orig* 

        # 修改个名字以便以后的操作
        ! 3dcopy blip_warp_Fwd.r{fwd}_WARP+orig blip_warp.r{fwd}_WARP+orig
        ! 3dcopy blip_warp_Rev.r{rev}_WARP+orig blip_warp.r{rev}_WARP+orig

        ! rm blip_warp_Fwd.r{fwd}+orig* blip_warp_Rev.r{rev}+orig* blip_warp_Fwd.r{fwd}_WARP+orig* blip_warp_Rev.r{rev}_WARP+orig* 

# for QC check，我们生成一个校正前和矫正后的平均EPI数据来作为distortion correction的效果证明
! 3dMean -prefix blip_pre_fwd.epi_mean.nii.gz rm.blip.med.masked.fwd.r*
! 3dMean -prefix blip_pre_rev.epi_mean.nii.gz rm.blip.med.masked.rev.r*
! 3dMean -prefix blip_post_fwd.epi_mean.nii.gz rm.blip.med.masked.fwd.post.r*
! 3dMean -prefix blip_post_rev.epi_mean.nii.gz rm.blip.med.masked.rev.post.r*

# remove redundent file
! rm rm*


files=files[:-3]
nRuns = len(files)
runstr=runstr[:-3]
for run in runstr:
    ! 3dNwarpApply -quintic -nwarp blip_warp.r{run}_WARP+orig      \
            -source pb02.{subj}.r{run}.tshift+orig         \
            -prefix pb03.{subj}.r{run}.blip


* 手动检查 1
   
    这四个文件也记录了对侧矫正的效果，只要有矫正效果，没有什么大问题即可。
  
   * blip_pre_fwd.epi_mean.nii.gz
   * blip_pre_rev.epi_mean.nii.gz
   * blip_post_fwd.epi_mean.nii.gz
   * blip_post_rev.epi_mean.nii.gz  


In [ ]:
# ================================= volreg =================================
base = 'vr_base_min_outlier+orig' #在前面生成

# align each dset to base volume, then we separate whether we want to further align epi to anat
for run in runstr:
    # register each volume to the base image
    ! 3dvolreg -verbose -zpad 1 -base {base} \
             -1Dfile dfile.r{run}.1D \
             -cubic \
             -1Dmatrix_save mat.r{run}.vr.aff12.1D \
             -prefix rm.epi.nomask.r{run} pb03.{subj}.r{run}.blip+orig

    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb03.{subj}.r{run}.blip+orig -expr 1 -prefix rm.epi.all1

    # warp the all-1 dataset for extents masking
    ! 3dAllineate -base {base} \
                -input rm.epi.all1+orig \
                -1Dmatrix_apply mat.r{run}.vr.aff12.1D \
                -final NN -quiet \
                -prefix rm.epi.1.r{run}

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+orig

# --------------- deal with motion parameter -------------------
import numpy as np
import matplotlib.pyplot as plt
# we take dmeaned motion (6), demean motion derivative (6) and their squares (12)
# 合并 motion params
! cat dfile.r*.1D > dfile_rall.1D
# 计算去均值的motion parameters (for use in regression)
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} -demean -write motion_demean.1D
# 计算一阶导数 (just to have)
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} -derivative -demean -write motion_deriv.1D
# calculate the square of demean and derivative of motion parameters (just to have)
# for resting preproc, we usually have 24 motion regressor (6motion+6deriv+12 their square)
np.savetxt('motion_demeansq.1D', np.loadtxt('motion_demean.1D')**2)
np.savetxt('motion_derivsq.1D', np.loadtxt('motion_deriv.1D')**2)
# create censor file motion_${subj}_censor.1D, for censoring motion
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} \
   -show_censor_count -censor_prev_TR \
   -censor_motion {motion_censor} motion_{subj}

# Estimate the motion parameter after motion correction, we can check the
# effects of MC
! mkdir mc
for run in runstr:
    ! 3dvolreg -verbose -zpad 1 -base {base} \
             -1Dfile dfile.r{run}_pos.1D \
             rm.epi.nomask.r{run}+orig
    ! rm volreg+orig*
# make a single file of motion params
! cat dfile.r*_pos.1D > dfile_rall_pos.1D

# make figure pre mc
dfile_pre = np.loadtxt('dfile_rall.1D')
plot(range(dfile_pre.shape[0]), dfile_pre[:,:3], color=['C0','C1','C2'], label=['roll(IS)','pitch(RL)','yaw(AP)'])
plt.legend();plt.xlabel('time points');plt.ylabel('mm');
plt.savefig('rots_pre.pdf');plt.close('all')
plot(range(dfile_pre.shape[0]), dfile_pre[:,3:], color=['C3','C4','C5'], label=['dS','dL','dP'])
plt.legend();plt.xlabel('time points (TR)');plt.ylabel('mm');
plt.savefig('tran_pre.pdf');plt.close('all')
# make figure pos mc
dfile_pos = np.loadtxt('dfile_rall_pos.1D')
plot(range(dfile_pos.shape[0]), dfile_pos[:,:3], color=['C0','C1','C2'], label=['roll(IS)','pitch(RL)','yaw(AP)'])
plt.legend();plt.xlabel('time points');plt.ylabel('mm');
plt.savefig('rots_pos.pdf');plt.close('all')
plot(range(dfile_pos.shape[0]), dfile_pos[:,3:], color=['C3','C4','C5'], label=['dS','dL','dP'])
plt.legend();plt.xlabel('time points (TR)');plt.ylabel('mm');
plt.savefig('tran_pos.pdf');plt.close('all')
# move file to directory
! mv rots*.pdf tran*.pdf dfile.r*_pos.1D dfile_r*_pos.1D mc/

# note TRs that were not censored, note ktrs here is a str
ktrs = unix_wrapper(f'1d_tool.py -infile motion_{subj}_censor.1D \
                       -show_trs_uncensored encoded', wantreturn=True, verbose=0)

# ----------------------------------------
# create the extents mask: mask_epi_extents+orig and apply the task
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
! 3dcalc -a rm.epi.mean+orig -expr "step(a-0.999)" -prefix mask_epi_extents
# and apply the extents mask to the EPI data
# (delete any time series with missing data)
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+orig -b mask_epi_extents+orig \
           -expr "a*b" -prefix pb04.{subj}.r{run}.volreg
! rm -f rm.*  # rm.epi.nomask are big files, remove them
! rm -f {base}* 

# calculate a mean epi volume for next step anat epi registration
! 3dMean -prefix rm.epi_mean.nii.gz pb04.{subj}.r*.volreg+orig.HEAD
! 3dTstat -prefix epi_mean.nii.gz rm.epi_mean.nii.gz
! rm rm* mask_epi_extents+orig*

# 删掉上一步distortion correction的数据，节省空间
! rm pb03*

* 手动检查 2
  
  打开mc/中的四个pdf，检查校正前后的头动情况  
  tran_pre不超过±3mm，tran_pos数量级大概是$10^{-2}$

In [ ]:
# ================================= anat2epi =================================
# 首先把用来配准的epi像make a copy
! 3dcopy epi_mean.nii.gz epi_mean_tmp.nii.gz

# 关键步骤。我们是把结构像配准到功能像，但是我们得到的转换矩阵是从相反的，功能像到结构像，这个矩阵可以后面用来转化epi数据
! align_epi_anat.py -anat2epi \
    -anat {t1ss} -anat_has_skull no \
    -suffix _al_junk \
    -epi epi_mean_tmp.nii.gz -epi_base 0 \
    -epi_strip 3dAutomask -giant_move \
    -volreg off -tshift off

# 生成nifti文件，方便我们在fsleyes上面查看配准效果
! 3dcopy {t1ss.pstem}_al_junk+orig {t1ss.pstem}_al_junk.nii.gz

# note that we align anat 2 epi, however, this xfm is from epi space to anat space
# note that this xfm is compatible with LPS+ space, which is the default space in AFNI
! mv {t1ss.pstem}_al_junk_mat.aff12.1D mat.anat2epi.aff12.1D

#######去头皮后的结构像就是t1ss({subj}_SurfVol_SS.nii), 不需要再生成
# create an skull-striped anat_final dataset, aligned with stats
#! 3dcopy {t1ss.pstem}_ns+orig anat_final.{subj}.nii.gz
#! rm {t1ss.pstem}_ns+orig*
! 3dcopy {t1ss} anat_final.{subj}.nii.gz
########

# rewrite the name of aligned anat
! 3dcopy {t1ss.pstem}_al_junk+orig anat2epi.{subj}.nii.gz
! rm {t1ss.pstem}_al_junk+orig*

# Invert xfm
! cat_matvec -ONELINE mat.anat2epi.aff12.1D -I > mat.epi2anat.aff12.1D

# warp the volreg base EPI dataset back to anat to make a final version
! 3dAllineate -base anat_final.{subj}.nii.gz \
            -input epi_mean.nii.gz \
            -1Dmatrix_apply mat.epi2anat.aff12.1D \
            -prefix epi2anat.{subj}.nii.gz

# Record final registration costs
! 3dAllineate -base epi2anat.{subj}.nii.gz -allcostX -input anat_final.{subj}.nii.gz > out.allcostX.txt

# Take the snapshots to show the quality of alignment
! @snapshot_volreg epi2anat.{subj}.nii.gz anat_final.{subj}.nii.gz
! @snapshot_volreg anat_final.{subj}.nii.gz epi2anat.{subj}.nii.gz
! @snapshot_volreg anat2epi.{subj}.nii.gz epi_mean.nii.gz
! @snapshot_volreg epi_mean.nii.gz anat2epi.{subj}.nii.gz


* 手动检查 3
   
   仔细检查配准结果,检查两者是否一致。

   * Underlay: epi_mean_tmp+orig  
   * Overlay: subj_SurfVol_al_junk+orig  

   如果不匹配，需要重新进行anat2epi的过程。开始之前，需要先删除上面产生的文件: 
      ```
      ! rm {sub}_SurfVol_al* {subj}_SurfVol_ns*
      ```
   确认配准没有问题之后，进行下一步操作



In [ ]:
# ================================= epi to individual anat =================================
# resolution=2.5  # 改到最开始的参数设置
# ======== Transform epi to match anat, add by RZ ===============
# In this step we need to concatenate Distortion(opt)+motion+affine transformations
# align each dset to base volume, then we separate whether we want to further align epi to anat
for run in runstr:
    # concatenate MC+aff transforms
    ! cat_matvec -ONELINE mat.anat2epi.aff12.1D -I mat.r{run}.vr.aff12.1D > mat.r{run}.warp.aff12.1D

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master {t1ss} -dxyz {resolution} \
            -source pb02.{subj}.r{run}.tshift+orig         \
            -prefix rm.epi.nomask.r{run}'
    unix_wrapper(cmd)

    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb02.{subj}.r{run}.tshift+orig -expr 1 -prefix rm.epi.all1

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master {t1ss} -dxyz {resolution} \
            -ainterp NN -quiet \
            -source rm.epi.all1+orig \
            -prefix rm.epi.1.r{run}'
    unix_wrapper(cmd)

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+orig
    # below file is big, should be removed
    ! rm rm.epi.1.r{run}+orig*

# ----------------------------------------
# create the extents mask: mask_epi_extents+orig
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
# and apply the extents mask to the EPI data
! 3dcalc -a rm.epi.mean+orig -expr "step(a-0.999)" -prefix mask_epi_extents

# 去掉mask之外的voxel的数据
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+orig -b mask_epi_extents+orig \
           -expr "a*b" -prefix pb05.{subj}.r{run}.al2anat
    # save to nifti format seems to reduce file size

# rm.epi.nomask are big files, remove them
! rm rm.*




# ================================= epi volume to surface =================================
surface_dir = FREESURFER_HOME+f'/subjects/{subj}/SUMA'
# map volume data to the surface of each hemisphere
for hemi in ['lh', 'rh']:
    for  run in runstr:
        ! 3dVol2Surf -spec {surface_dir}/std.141.{subj}_{hemi}.spec   \
                   -sv {subj}_SurfVol_2std.nii           \
                   -surf_A smoothwm                            \
                   -surf_B pial                                \
                   -f_index nodes                              \
                   -f_steps 10                                 \
                   -map_func ave                               \
                   -oob_value 0                                \
                   -grid_parent pb05.{subj}.r{run}.al2anat+orig   \
                   -out_niml pb05.{subj}.{hemi}.r{run}.surf.niml.dset



# ================================= epi to MNI anat =================================
# ======== Transform epi to match anat, add by RZ ===============
# In this step we need to concatenate Distortion(opt)+motion+affine transformations
# align each dset to base volume, then we separate whether we want to further align epi to anat
for run in runstr:

    # we apply distortion correction + motion correction + anat2epi + nonlinear warp to MNI transformation
    # 注意我们这里是相反的顺序来concatenate的
    cmd = f'3dNwarpApply -quintic -nwarp "anatQQ.{subj}_WARP.nii anatQQ.{subj}.aff12.1D mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master MNI152_2009_template_SSW.nii.gz \
            -dxyz {resolution} \
            -source pb02.{subj}.r{run}.tshift+orig \
            -prefix rm.epi.nomask.r{run}'
    unix_wrapper(cmd)

    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb02.{subj}.r{run}.tshift+orig -expr 1 -prefix rm.epi.all1

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "anatQQ.{subj}_WARP.nii anatQQ.{subj}.aff12.1D mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master MNI152_2009_template_SSW.nii.gz \
            -dxyz {resolution} \
            -source rm.epi.all1+orig         \
            -ainterp NN -quiet \
            -prefix rm.epi.1.r{run}'
    unix_wrapper(cmd)

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+tlrc
    # below file is big, should be removed
    ! rm rm.epi.1.r{run}+tlrc*      


# ----------------------------------------
# create the extents mask: mask_epi_extents+orig
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
! 3dcalc -a rm.epi.mean+tlrc -expr "step(a-0.999)" -prefix mask_epi_2mni_extents

# 去掉mask之外的voxel的数据
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+tlrc -b mask_epi_2mni_extents+tlrc \
           -expr "a*b" -prefix pb06.{subj}.r{run}.al2mni
    # save to nifti format seems to reduce file size

# rm.epi.nomask are big files, remove them
! rm rm.*


# ----------------------------------------
# smooth
# 补充步骤，用于进行smooth操作，  2022.7.3添加
# 目的是对上一步step8所生成的pb06进行smooth   生成pb06s
# 再对上上一步step7所生成的pb05***surf.niml文件进行smooth  生成pb05s*rh  和 pb05s*lh
# 再对上上一步step7所生成的pb05.subj.r0*.al2anat+orig文件进行smooth  生成pb05s*.subj.r0*.blur+al2anat+orig

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb06s.{subj}.r{run}.blur+al2mni+tlrc \
        pb06.{subj}.r{run}.al2mni+tlrc

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.lh.r{run}.blur.surf.niml.dset \
        pb05.{subj}.lh.r{run}.surf.niml.dset

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.rh.r{run}.blur.surf.niml.dset \
        pb05.{subj}.rh.r{run}.surf.niml.dset

#以下代码可以不用跑
for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.r{run}.blur+al2anat+orig \
        pb05.{subj}.r{run}.al2anat+orig




In [ ]:
# ====================== Mask =================================
# Create 'full_mask' dataset (union mask)
for run in runstr:
    ! 3dAutomask -prefix rm.mask_r{run}.nii.gz pb06.{subj}.r{run}.al2mni+tlrc

# 创造一个所有功能像的run合起来的mask
! 3dmask_tool -inputs rm.mask_r*.nii.gz -union -prefix full_mask.{subj}.nii.gz
! rm rm.mask*

# ---- create subject anatomy mask, mask_anat.$subj+orig ----
#      (resampled from aligned anat)
# 把功能像resample到上面mask的分辨率
! 3dresample -master full_mask.{subj}.nii.gz -input \
           MNI152_2009_template_SSW.nii.gz -prefix rm.resam.anat.nii.gz
# convert to binary anat mask; fill gaps and holes
! 3dmask_tool -dilate_input 5 -5 -fill_holes -input rm.resam.anat.nii.gz \
            -prefix mask_anat.{subj}.nii.gz

# 结合功能像和结构像的mask
# compute tighter EPI mask by intersecting with anat mask
! 3dmask_tool -input full_mask.{subj}.nii.gz mask_anat.{subj}.nii.gz \
            -inter -prefix mask_epi_anat.{subj}.nii.gz
# note Dice coefficient of masks, as well
! 3ddot -dodice full_mask.{subj}.nii.gz mask_anat.{subj}.nii.gz > out.mask_ae_dice.txt

# 删掉多余文件
! rm rm*




# ====================== scale =================================
# 注意！！！这个cell的代码是对pb06以及pb05 进行scale重置 也就是未经smooth的数据
# scale each voxel time series to have a mean of 100
# (be sure no negatives creep in)
# (subject to a range of [0,200])
# scale volume data

for run in runstr:
    ! 3dTstat -prefix rm.mean_r{run} pb06.{subj}.r{run}.al2mni+tlrc
    ! 3dcalc -a pb06.{subj}.r{run}.al2mni+tlrc -b rm.mean_r{run}+tlrc \
           -c mask_epi_2mni_extents+tlrc \
           -expr "c * min(200, a/b*100)*step(a)*step(b)" \
           -prefix pb07.{subj}.r{run}.scale

# combine all datasets into one
! 3dTcat -prefix all_runs.{subj}.nii.gz pb07.{subj}.r*.scale+tlrc*
# remove redundant file
! rm rm.mean*

# 再scale surface data
for hemi in ['lh', 'rh']:
    for run in runstr:
       ! 3dTstat -prefix rm.{hemi}.mean_r{run}.niml.dset    \
            pb05.{subj}.{hemi}.r{run}.surf.niml.dset
       ! 3dcalc -a pb05.{subj}.{hemi}.r{run}.surf.niml.dset  \
               -b rm.{hemi}.mean_r{run}.niml.dset          \
               -expr 'min(200, a/b*100)*step(a)*step(b)' \
               -prefix pb07.{subj}.{hemi}.r{run}.scale.niml.dset

# remove redundant file
! rm rm.*mean* 


# 注意！！！这个cell的代码是对pb06s 以及pb05s 进行scale重置 也就是经过了smooth的数据
# 生成pb07s*.scale  
# 这个文件被整合成all_runs_with_smooth
# 生成pb07s*.scale.niml.dset

# scale volume data
for run in runstr:
    ! 3dTstat -prefix rm.mean_r{run} pb06s.{subj}.r{run}.blur+al2mni+tlrc
    ! 3dcalc -a pb06s.{subj}.r{run}.blur+al2mni+tlrc -b rm.mean_r{run}+tlrc \
           -c mask_epi_2mni_extents+tlrc \
           -expr "c * min(200, a/b*100)*step(a)*step(b)" \
           -prefix pb07s.{subj}.r{run}.scale

# combine all datasets into one
! 3dTcat -prefix all_runs_with_smooth.{subj}.nii.gz pb07s.{subj}.r*.scale+tlrc*
# remove redundant file
! rm rm.mean*

# 再scale surface data
for hemi in ['lh', 'rh']:
    for run in runstr:
       ! 3dTstat -prefix rm.{hemi}.mean_r{run}.niml.dset    \
            pb05s.{subj}.{hemi}.r{run}.blur.surf.niml.dset
       ! 3dcalc -a pb05s.{subj}.{hemi}.r{run}.blur.surf.niml.dset  \
               -b rm.{hemi}.mean_r{run}.niml.dset          \
               -expr 'min(200, a/b*100)*step(a)*step(b)' \
               -prefix pb07s.{subj}.{hemi}.r{run}.scale.niml.dset
# remove redundant file
! rm rm.*mean* 

### funcpp03

In [ ]:
# ================================ regress =================================
# compute de-meaned motion parameters (for use in regression)
# in regress (detrend step)

# ------------------------------
# run the regression analysis. Note that stats results should be written as afni format

! 3dDeconvolve -input pb07.{subj}.r*.scale+tlrc.HEAD \
    -mask mask_epi_anat.{subj}.nii.gz \
    -censor motion_{subj}_censor.1D \
    -ortvec motion_demean.1D mot_demean \
    -polort 2 \
    -num_stimts 0 \
    -rout \
    -fout -tout -x1D X.xmat.1D -xjpeg X.jpg \
    -x1D_uncensored X.nocensor.xmat.1D \
    -fitts motpoly.fitts.{subj} \
    -errts motpoly.errts.{subj} \
    -bucket motpoly.stats.{subj}

# should add save a script here

# -- use 3dTproject to project out regression matrix --
! 3dTproject -polort 0 -input pb07.{subj}.r*.scale+tlrc.HEAD \
           -censor motion_{subj}_censor.1D -cenmode ZERO \
           -ort X.nocensor.xmat.1D -prefix motpoly.errts.{subj}.tproject
# 3dTproject can also supply '-passband 0.01, 0.08' to band pass time seriest

! rm -f motpoly.stats* motpoly.fitts* # remove some large files



# --------------------- SNR -----------------------------
# create a temporal signal to noise ratio dataset
#    signal: if 'scale' block, mean should be 100
#    noise : compute standard deviation of errts
unix_wrapper(f'3dTstat -mean -prefix rm.signal.all.nii.gz all_runs.{subj}.nii.gz"[{ktrs}]"')
unix_wrapper(f'3dTstat -stdev -prefix rm.noise.all.nii.gz motpoly.errts.{subj}.tproject+tlrc"[{ktrs}]"')
! 3dcalc -a rm.signal.all.nii.gz \
       -b rm.noise.all.nii.gz \
       -c full_mask.{subj}.nii.gz \
       -expr "c*a/b" -prefix TSNR.{subj}.nii.gz

# remove redundant files
! rm rm.signal.all* rm.noise.all*
! rm motpoly.errts* X*

In [ ]:
# ========================== auto block: finalize ==========================
# Remove temporary files
! rm -f rm.* # finally remove some left-over files

# We can further remove despike, tshift and volreg data at the end volreg are big
! rm -f all_runs*
! rm anatQQ* MNI152_2009_template_SSW.nii.gz
! rm pb01* pb04*
! rm epi_mean_tmp*

# organize data


# ==================== preprocessing done ==================================
# return to previous directory
os.chdir(cwd)
os.environ['SHELL'] = orig_shell     # switch back to the original shell
print(f'\n=============== Preprocessing started: {start_time} ================\n')
print(f'\n=============== Preprocessing finished: {gettimestr("full")} ================\n')

### 手动检查

1. **Distortion Correction**  
   
    这四个文件也记录了对侧矫正的效果，只要有矫正效果，没有什么大问题即可。
  
   * blip_pre_fwd.epi_mean.nii.gz
   * blip_pre_rev.epi_mean.nii.gz
   * blip_post_fwd.epi_mean.nii.gz
   * blip_post_rev.epi_mean.nii.gz  

>

2. **Motion Correction**
   
   打开mc/中的四个pdf，检查校正前后的头动情况  
   tran_pre不超过±3mm，tran_pos数量级大概是$10^{-2}$

>

3. **anat2epi**
   
   仔细检查配准结果,检查两者是否一致。

   * Underlay: epi_mean_tmp+orig  
   * Overlay: subj_SurfVol_al_junk+orig  

   如果不匹配，需要重新进行anat2epi的过程。开始之前，需要先删除上面产生的文件: 
      ```
      ! rm {sub}_SurfVol_al* {subj}_SurfVol_ns*
      ```
   确认配准没有问题之后，进行下一步操作